# Lesson 3: Chatbot 예제

이 수업에서는 코스 전반에 걸쳐 작업할 chatbot 예제를 살펴보겠습니다. 예제에는 tool 정의 및 실행과 chatbot 코드가 포함되어 있습니다. notebook 마지막에서 chatbot과 상호작용해 보세요.

## Library Import

In [2]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
from google import genai
from google.genai import types

## Tool Function들

In [3]:
PAPER_DIR = "papers"

첫 번째 tool은 주제를 기반으로 관련 arXiv paper를 검색하고 해당 논문들의 정보(제목, 저자, 요약, PDF URL, 발행일)를 JSON 파일에 저장합니다. JSON 파일들은 `papers` 디렉토리 내에서 주제별로 구성됩니다. 이 tool은 논문을 다운로드하지는 않습니다.

In [4]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

In [5]:
search_papers("computers")

Results are saved in: papers/computers/papers_info.json


['1310.7911v2',
 'math/9711204v1',
 '2208.00733v1',
 '2504.07020v1',
 '2403.03925v1']

두 번째 tool은 `papers` 디렉토리 내의 모든 주제 디렉토리에서 특정 논문에 대한 정보를 찾습니다.

In [6]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."

In [7]:
extract_info('1310.7911v2')

'{\n  "title": "Compact manifolds with computable boundaries",\n  "authors": [\n    "Zvonko Iljazovic"\n  ],\n  "summary": "We investigate conditions under which a co-computably enumerable closed set\\nin a computable metric space is computable and prove that in each locally\\ncomputable computable metric space each co-computably enumerable compact\\nmanifold with computable boundary is computable. In fact, we examine the notion\\nof a semi-computable compact set and we prove a more general result: in any\\ncomputable metric space each semi-computable compact manifold with computable\\nboundary is computable. In particular, each semi-computable compact\\n(boundaryless) manifold is computable.",\n  "pdf_url": "http://arxiv.org/pdf/1310.7911v2",\n  "published": "2013-10-29"\n}'

## Tool Schema

LLM에 제공할 각 tool의 schema입니다.

In [8]:
gemini_tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arXiv based on a topic and store their information.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for"
                }, 
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "parameters": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to look for"
                }
            },
            "required": ["paper_id"]
        }
    }
]

# Create tools for Gemini
tools = types.Tool(function_declarations=gemini_tools)

## Tool Mapping

이 코드는 tool mapping과 실행을 처리합니다.

In [9]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."
        
    elif isinstance(result, list):
        result = ', '.join(result)
        
    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)
    
    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot 코드

Chatbot은 사용자의 query를 하나씩 처리하지만, query들 간에 메모리를 유지하지는 않습니다.

In [10]:
load_dotenv()

# Set the API key from environment variable
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', '')

client = genai.Client()

### Query 처리

In [11]:
def process_query(query):
    
    config = types.GenerateContentConfig(tools=[tools])
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=query,
        config=config
    )
    
    process_query = True
    while process_query:
        
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'text') and part.text:
                print(part.text)
                process_query = False
                break
            
            elif hasattr(part, 'function_call') and part.function_call:
                
                function_call = part.function_call
                tool_name = function_call.name
                tool_args = dict(function_call.args)
                
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                result = execute_tool(tool_name, tool_args)
                
                # Continue conversation with function response
                function_response = types.FunctionResponse(
                    name=tool_name, 
                    response={"result": result}
                )
                
                response = client.models.generate_content(
                    model='gemini-2.5-flash',
                    contents=[
                        types.Content(role='user', parts=[types.Part(text=query)]),
                        types.Content(role='model', parts=[types.Part(function_call=function_call)]),
                        types.Content(role='function', parts=[types.Part(function_response=function_response)])
                    ],
                    config=config
                )
                
                # Print the final response
                if response.candidates[0].content.parts:
                    for final_part in response.candidates[0].content.parts:
                        if hasattr(final_part, 'text') and final_part.text:
                            print(final_part.text)
                            break
                process_query = False

### Chat Loop

In [12]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Chatbot과 자유롭게 상호작용해 보세요. 예시 query:

- "LLM interpretability"에 관한 논문 2편 검색해줘

In [13]:
chat_loop()

Type your queries or 'quit' to exit.
Calling tool search_papers with args {'topic': 'ai'}
Results are saved in: papers/ai/papers_info.json
네, AI 관련 논문 다섯 편을 찾았습니다. 제목과 초록 등 자세한 정보를 원하시면 논문의 ID를 알려주세요.


어떤 정보에 대한 ID를 찾고 계신가요? 찾으시는 정보의 주제를 알려주시면 관련 논문을 검색하여 ID를 알려드릴 수 있습니다.


Calling tool search_papers with args {'topic': 'ai'}
Results are saved in: papers/ai/papers_info.json
AI 관련 논문들의 ID는 다음과 같습니다: 2409.12922v1, 2406.11563v3, 2402.07632v3, 2211.05075v1, 2403.15481v2.


Calling tool extract_info with args {'paper_id': '2409.12922v1'}
"AI Thinking: A framework for rethinking artificial intelligence in practice" 논문에 대한 정보를 찾았습니다. 

**저자**: Denis Newman-Griffis
**발행일**: 2024년 8월 26일
**요약**: 이 논문은 인공지능이 정보를 다루는 방식을 변화시키고 있으며, AI 사용에 대한 다양한 학문 분야의 이해를 통합할 필요성을 강조합니다. 저자는 AI 사용과 관련된 주요 결정 및 고려 사항을 모델링하는 AI 사고라는 새로운 개념적 프레임워크를 제안합니다. 이 모델은 정보 프로세스에서 AI 사용 동기 부여, AI 방법 공식화, 사용 가능한 도구 및 기술 평가, 적절한 데이터 선택, AI를 사용되는 사회 기술적 맥락에 배치하는 다섯 가지 실천 기반 역량을 다룹니다.

자세한 내용은 다음 PDF에서 확인하실 수 있습니다: http://arxiv

다음 수업에서는 tool 정의를 추출하여 MCP server로 래핑할 것입니다. 그런 다음 chatbot 내부에 MCP client를 생성하여 chatbot을 MCP 호환으로 만들 것입니다.